In [1]:
import pandas as pd
import dtale
import FundamentalAnalysis as fa
import yfinance as yf
import finviz
from datetime import date
from datetime import datetime
from dateutil import relativedelta

In [2]:
def insider_date(symbol):
    for i in finviz.get_insider(symbol):
        tradedate = [i['Date'] + ' 2021']
        insiderdf = pd.DataFrame.from_dict(i)
        tradedate = pd.to_datetime(tradedate, format = '%b %d %Y')
    return tradedate[0]

In [3]:
def get_finviz(symbol, date):
    finStock = finviz.get_insider(symbol)
    finStock = pd.DataFrame.from_dict(finStock)
    dtFinStock = pd.to_datetime(finStock['Date'], format = '%b %d')
    finStock['Date'] = dtFinStock
    today = pd.to_datetime(date.today())
    thisYear = finStock['Date'].loc[finStock['Date'].dt.strftime('%m-%d') <= today.strftime('%m-%d')]
    updatedDates = [None for x in range(finStock['Date'].size)]
    count = 0
    for date in thisYear:
        thisYear = date.replace(2021)
        updatedDates[count] = thisYear
        count = count + 1


    lastYear = finStock['Date'].loc[finStock['Date'].dt.strftime('%m-%d') > today.strftime('%m-%d')]
    for date in lastYear:
        lastYear = date.replace(2020)
        updatedDates[count] = lastYear
        count = count + 1


    finStock['Date'] = updatedDates
    finStock.set_index('Date', inplace=True)
    return finStock

In [4]:
def six_month_insider(symbol):
    try:
        finDf = get_finviz(symbol, date)
        idxSixMonth = finDf[datetime.today() : datetime.today() - relativedelta.relativedelta(months=6)]
        sixMonthValue = idxSixMonth['Value ($)']
        sixMonthValue = pd.to_numeric(sixMonthValue.str.replace(',',''), errors='coerce')
        transaction = idxSixMonth['Transaction']
        sixMonthValue[transaction == 'Sale'] = -sixMonthValue[transaction == 'Sale'] 
        sixMonthValue[transaction == 'Option Exercise'] = -sixMonthValue[transaction == 'Option Exercise'] 
        sixMonthSum = sixMonthValue.sum(axis = 0)
    except:
        sixMonthSum = 0
    return sixMonthSum

In [5]:
def one_month_insider(symbol):
    try:
        finDf = get_finviz(symbol, date)
        idxOneMonth = finDf[datetime.today() : datetime.today() - relativedelta.relativedelta(months=1)]
        oneMonthValue = idxOneMonth['Value ($)']
        oneMonthValue = pd.to_numeric(oneMonthValue.str.replace(',',''), errors='coerce')
        transaction = idxOneMonth['Transaction']
        oneMonthValue[transaction == 'Sale'] = -oneMonthValue[transaction == 'Sale'] 
        oneMonthValue[transaction == 'Option Exercise'] = -oneMonthValue[transaction == 'Option Exercise'] 
        oneMonthSum = oneMonthValue.sum(axis = 0)
    except:
        oneMonthSum = 0
    return oneMonthSum

In [6]:
def one_week_insider(symbol):
    try:
        finDf = get_finviz(symbol, date)
        idxOneWeek = finDf[datetime.today() : datetime.today() - relativedelta.relativedelta(week=1)]
        oneWeekValue = idxOneWeek['Value ($)']
        oneWeekhValue = pd.to_numeric(oneWeekValue.str.replace(',',''), errors='coerce')
        transaction = idxOneWeek['Transaction']
        oneWeekValue[transaction == 'Sale'] = -oneWeekValue[transaction == 'Sale'] 
        oneWeekValue[transaction == 'Option Exercise'] = -oneWeekValue[transaction == 'Option Exercise'] 
        oneWeekSum = oneWeekValue.sum(axis = 0)
    except:
        oneWeekSum = 0
    return oneWeekSum

In [18]:
ticker = yf.Tickers('msft aapl amzn pltr amd psfe gte pbf')
stonk = pd.DataFrame()
count = 0
for symbol in ticker.tickers:
    dateSymbol = symbol
    symbol = ticker.tickers[symbol]
    dfList = [symbol.info['shortName'],
              symbol.info['currentPrice'],
              (symbol.info['currentPrice'] - symbol.info['open'])
              /symbol.info['open']*100,
              symbol.info['52WeekChange']*100,
              symbol.info['marketCap'] / 1e9,
              symbol.info['totalRevenue'] / 1e9,
              #symbol.info['shortPercentOfFloat']*100,
              symbol.info['pegRatio'],
              # ticker.info['trailingPE'],
              symbol.info['trailingEps'],
              symbol.info['forwardEps'],
              symbol.info['priceToSalesTrailing12Months'],
              symbol.info['enterpriseToEbitda'],
              symbol.info['totalCashPerShare'],
              symbol.info['revenueGrowth']*100,
              one_week_insider(dateSymbol) / 1e6,
              one_month_insider(dateSymbol) / 1e6,
              six_month_insider(dateSymbol) / 1e6]
    ratiodf = pd.DataFrame(dfList)
    count = count + 1
    ratiodf = ratiodf.T
    ratiodf.columns = ['Company Name', 'Price','Day %', '52 Week Change (%)', 'Market Cap (b)', 'Revenue (b)', #'Short %',
                           'PEG', 'EPS (TTM)', 'EPS (NTM)', 'P/S (TTM)',
                           'EV/EBITDA', 'Cash Per Share', 'Revenue Growth (%)','1w Insider Sales (m)','1m Insider Sales (m)','6m Insider Sales (m)']
    stonk = pd.concat([stonk, ratiodf])

dtale.show(stonk)

In [8]:
# Outstanding shares, Rev, Rev ava3, turnover, gross income/tq, cfebit adj/ tq, roic ava,
#gross margin, ebitda margin, net inc margin, cfo margin, sfcf margin, ncf margin, revs/sh,
#assets/sh, net excess cash, net com over, book/sh, tang book/sh, bm roe 3yr %, bm roe 5yr %,
# tbm ROE 3yr %, tbm roe 5yr %, div/sh, bm EPS ava, EBITDA/sh, 

In [23]:
ticker = 'AMD'
api_key = ''
income_statement_annually = fa.income_statement(ticker, api_key, period="annual")
key_metrics_annually = fa.key_metrics(ticker, api_key, period="annual")
financial_ratios_annually = fa.financial_ratios(ticker, api_key, period="annual")

In [24]:
def make_fundamental_sheet(df):
    dfDf = pd.DataFrame()
    for i in range(len(df)):
        dfNum = pd.to_numeric(df.iloc[i], errors='coerce')
        dfNum = pd.DataFrame(dfNum)
        dfNum = dfNum.T
        dfDf = pd.concat([dfDf, dfNum])
    return dfDf

In [25]:
km = key_metrics_annually.loc[['cashPerShare','revenuePerShare','netIncomePerShare',
                               'roic','roe','dividendYield']]
    
ins = income_statement_annually.loc[['weightedAverageShsOut','revenue']] / 1e9
filingDate = pd.DataFrame(income_statement_annually.loc['fillingDate'])
filingDate = filingDate.T
 
fr = financial_ratios_annually.loc[['grossProfitMargin','netProfitMargin',
                                    'assetTurnover','operatingCashFlowPerShare',
                                    'freeCashFlowPerShare','cashPerShare',
                                    'priceEarningsRatio','debtRatio']]

insDf = make_fundamental_sheet(ins)
kmDf = make_fundamental_sheet(km)
frDf = make_fundamental_sheet(fr)
model = pd.concat([insDf, kmDf, frDf])
modelSheet = model.reset_index().style.background_gradient(axis=1, cmap = 'YlGnBu')
modelSheet

,index,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990,1989,1988,1987,1986
0,weightedAverageShsOut,1.184000,1.120000,1.064000,0.952000,0.835000,0.783000,0.768000,0.754000,0.741000,0.727000,0.733000,0.678000,0.607000,0.558000,0.492000,0.440800,0.371100,0.346900,0.342300,0.332400,0.309331,0.294600,0.287800,0.280900,0.270300,0.211150,0.209100,0.190216,0.181860,0.175195,0.164103,0.162273,0.148333,0.115542,0.110909
1,revenue,9.763000,6.731000,6.475000,5.253000,4.272000,3.991000,5.506000,5.299000,5.422000,6.568000,6.494000,5.403000,5.808000,6.013000,5.649000,5.847600,5.001400,3.519200,2.697000,3.891800,4.644187,2.857600,2.542100,2.356400,1.953000,2.429724,2.134700,1.648280,1.514500,1.226600,1.059200,1.104600,1.125900,0.632000,0.576100
2,cashPerShare,1.934122,1.341964,1.086466,1.244748,1.513772,1.002554,1.354167,1.454907,1.352227,2.427785,2.440655,3.946903,1.805601,3.385305,3.132114,4.072142,3.220156,3.787835,3.032428,2.617329,4.180522,2.023082,2.421821,1.662513,1.428043,2.323689,1.807747,2.390985,1.847716,1.575092,0.702076,0.491148,1.774892,1.511254,0.733157
3,revenuePerShare,8.245777,6.009821,6.085526,5.517857,5.116168,5.097063,7.169271,7.027851,7.317139,9.034388,8.859482,7.969027,9.568369,10.775986,11.481707,13.265880,13.477230,10.144710,7.879053,11.708183,15.013649,9.699932,8.832870,8.388750,7.225305,11.507099,10.208991,8.075943,7.687817,6.418629,6.466422,6.807059,6.962894,6.412219,5.472919
4,netIncomePerShare,2.103041,0.304464,0.316729,-0.034664,-0.595210,-0.842912,-0.524740,-0.110080,-1.596491,0.675378,0.642565,0.554572,-5.103789,-6.055556,-0.337398,0.375454,0.245756,-0.791294,-3.806602,-0.182310,3.177910,-0.301765,-0.361362,-0.075116,-0.255272,1.423258,1.460067,1.121019,1.243655,0.760335,-0.327228,0.284090,0.119357,-0.411576,-0.628798
5,roic,0.209167,0.124012,0.170684,0.056400,-0.193418,-0.286139,-0.092823,0.037514,-0.407222,0.146541,0.250456,0.122015,-0.358781,-0.321008,0.035843,0.042389,0.036979,-0.041368,-0.288533,-0.013455,0.195601,-0.092558,-0.051319,-0.032532,-0.098868,0.144541,0.245469,0.215604,0.248037,0.173596,-0.067506,0.045476,0.023681,-0.056858,-0.145416
6,roe,0.426589,0.120623,0.266193,-0.055369,-1.194712,1.601942,-2.155080,-0.152574,-2.198885,0.308805,0.464956,0.580247,37.780488,-1.130100,-0.027325,0.049374,0.030299,-0.112592,-0.528172,-0.017046,0.309940,-0.044922,-0.051870,-0.010394,-0.034125,0.143097,0.175965,0.169231,0.234002,0.185568,-0.084277,0.066725,0.029922,-0.102729,-0.186667
7,dividendYield,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000005,0.003183,0.005096,0.005380,0.005245,0.018470,0.017680,0.014097,0.010289,nan
8,grossProfitMargin,0.445252,0.426088,0.377915,0.340187,0.233614,0.270609,0.333999,0.373278,0.227776,0.447625,0.455959,0.420507,0.399449,0.376185,0.494424,0.409023,0.393670,0.338742,0.219281,0.334550,0.458541,0.312570,0.323905,0.330122,0.262263,0.465422,0.539842,0.520961,0.507098,0.462906,0.359422,0.541010,0.412115,0.435964,0.399952
9,netProfitMargin,0.255045,0.050661,0.052046,-0.006282,-0.116339,-0.165372,-0.073193,-0.015663,-0.218185,0.074756,0.072528,0.069591,-0.533402,-0.561949,-0.029386,0.028302,0.018235,-0.078001,-0.483129,-0.015571,0.211668,-0.031110,-0.040911,-0.008954,-0.035330,0.123685,0.143018,0.138810,0.161770,0.118458,-0.050604,0.041735,0.017142,-0.064186,-0.114893
